# HW1: MLS (bonus question)

In this exercise you will:

* Compute an implicit MLS function that approximates a cloud of 3D points with normals. The input points will then lie at the zero level set of the computed function.

* Sample the implicit function on a three dimensional volumetric grid.

* Reconstruct a triangle mesh of the zero level set of the implicit function, by using the Marching
Cubes algorithm on the grid.

* Experiment with various MLS reconstruction parameters.

## Part 0: overview

In [1]:
#enable plugin to automatically reload the module when a method is called
%reload_ext autoreload 
%autoreload 2
import numpy as np
import polyscope as ps
from utils import read_off, to_np, bounding_box_diag
from IPython.display import Image

In the data/ folder we have 3D models in .off format. Lets load ./data/cat.off and see what's there

In [2]:
data_path = './data/cat.off'
vertices, faces_gt, normals = read_off(data_path)
print(f"{vertices.shape=}")
print(f"{normals.shape=}")
print(f"{faces_gt.shape=}")

vertices.shape=(366, 3)
normals.shape=(366, 3)
faces_gt.shape=(698, 3)


Lets visualize the 3d model. We'll use the modern and powerful tool – [polyscope](https://polyscope.run/py/)

In [3]:
ps.init() # Initialization. Don't need to reinitialize each time
if 'cat' in data_path:
    ps.set_up_dir("z_up") # Fix default camera orientation 
gt_mesh = ps.register_surface_mesh("gt_mesh", vertices, faces_gt)
gt_mesh.set_enabled(True)

ps_cloud = ps.register_point_cloud('pts', vertices)
ps_cloud.add_vector_quantity("Normal vec", normals, radius=0.01, length=0.02, color=(0.2, 0.5, 0.5), enabled=True)

ps.show() # a window with visualization should be opened.

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83


GLFW emitted error: Cocoa: Failed to find service port for display


## Part 1: setting up constraints

The task of this assignment is to come up with an implicit function $f(p), p = (x,y,z)$ defined on all of the 3D space, such that the input point cloud lies at the zero level set of this function, i.e. for any point pi in the input point cloud, we have $f(p_i) = 0$. The normals of the implicit function evaluated at the point cloud locations should agree with the normals of the point cloud (also provided as input).

The implicit function is typically interpolated from a set of given values at given points. Given the
function values $f_i$ at some given points $p_i$ (the constrained points), the function $f$ is found by solving
a minimization of the form $f = \argmin_{\phi} \sum_{i} ||\phi(p_i) - f_i || $ , where $\phi(p)$ 
is any acceptable function (such as any function computed via an MLS procedure). 
The first step is thus specifying those constraints. Naturally, the input point cloud should be part of the constraints, 
with the value of $f = 0$. However, if the constraints only lie on the zero level set, most interpolation schemes will 
only generate the zero function as a result. We thus need to introduce more constraints. 
The process for generating the constraints is as follows:

• For all point cloud points $p_i$ we know the constraint $f(p_i)$ = 0.

• Fix an ε value, for instance $ε = 0.01 ×$ bounding box diagonal.

• For each point $p_i$ compute $\hat{p_{i}} = p_i + εn_i$, where $n_i$ is the normalized normal of $p_i$. Check
whether $\hat{p_i}$ is the closest point to $p_i$ ; if not, divide ε by 2 and recompute $\hat{p_i}$ until this is
the case. Then, add another constraint equation: $f (\hat{p_i} ) = ε$.

• Repeat the same process for −ε, i.e., add equations of the form $f(p^*_i) = −ε$. Dont forget
to check that $p^*_i$ is the closest point to p_i .

As a result of the steps, for the n point cloud points you get 3n equations for the
implicit function $f(x,y,z)$.

__Task 1__: 

Please implement `sample_constraints(...)` in `task1_constraints.py`

Once it is done, we can visualize the result:

In [4]:
from task1_constraints import sample_constraints
from tests import check_constraints

bbox_diag = bounding_box_diag(vertices)
eps = bbox_diag * 0.01 # define eps parameter for sampling = 0.01 * bounding box diagonal
new_verts, new_vals = sample_constraints(vertices, normals, eps)

all_pts = np.concatenate([vertices, new_verts])
all_vals = np.concatenate([np.zeros(len(vertices)), new_vals])

# Sampled constraints visualization
ps.register_point_cloud('pos pts', all_pts[all_vals>0])
ps.register_point_cloud('neg pts', all_pts[all_vals<0])

ps.show()

(732, 3) (732,)


The result should look like ~that:

<img src="./imgs/screenshot_000003.jpg" width="800" />

Run tests to check if it's correct

In [5]:
!pytest tests.py::test_constraints_sampling_3D

============================= test session starts ==============================
platform darwin -- Python 3.10.9, pytest-7.2.1, pluggy-1.0.0
rootdir: /Users/samp8/PycharmProjects/hw1-amoazeni75
plugins: anyio-3.6.2
collected 1 item                                                               

tests.py .                                                               [100%]

============================== 1 passed in 0.57s ===============================


## Part 2: MLS Interpolation

Given the constraints, you can use interpolation to construct the implicit function. The function is not defined explicitly e.g. by a formula; instead it can be evaluated at any given point in 3D space. At any point, the function is computed as a weighted sum of polynomial functions defined on the constrained points, where the weights vary from one points to the other (MLS).

More specifically:

2.1 First, define a grid in space that contains the constraint points from part 1. The resolution of the grid - is a parameter to be adjusted in experiments. (it's already implemented for you)

2.2 For each node of the grid compute the value of the implicit function f(x,y,z) whose zero level set approximates the point cloud. Use the moving least squares approximation

* For each grid node find closest constraint vertices within a fixed radius. Check if the number of them is sufficient to build a polynomial of chosen degree 

* Find parameters of a polynomial that approximates the values of the constraint vertices
    
* Evaluate the polynomial at the grid node.


This part contains 4 tasks:


 __Task 2__: 
 Please implement the missing part of `predict_grid(...)` function in `task2_solver.py`. Specifically, you need to find closest constraint points in a fixed radius (local_radius). Note, that you should also specify the minimum number of neighbours required to build the polynomial of the specified degree 

 __Task 3__: 
 Please implement the missing part of `eval_grid_point(...)` function in `task2_solver.py`. Given constrain points and their values you need to build a mathematical model that maps a 3D point position to a value. Here, we propose you to use the polynomial model (max degree == 2). Its coefficients can be found via least squares optimization over constrain points. 

#### 2.1 Grid in space 

In [6]:
from task2_solver import generate_grid

resolution = 30
grid_pts, coords_matrix = generate_grid(all_pts, resolution)
local_radius = bbox_diag * 0.1

ps.register_point_cloud('grid pts', grid_pts, radius=0.001)
ps.show()

#### 2.2 Evaluate implicit function at the grid points

In [7]:
from task2_solver import local_predictor
from utils import vals2colors

pred_vals = local_predictor(
        grid_pts=grid_pts,
        constr_pts=all_pts,
        constr_vals=all_vals,
        local_radius=local_radius,
        degree=1,
        reg_coef=1)

colors = vals2colors(pred_vals) # map implicit value to color for visualization
grid_cloud = ps.register_point_cloud('grid pts', grid_pts, radius=0.001)
grid_cloud.add_color_quantity("rand colors", colors, enabled=True)
ps.show()

Output should look like that:

<img src="imgs/screenshot_000000.jpg" width="800" />

## Part 3: Extracting the surface

You can now use marching cubes to extract the zero isosurface from your grid. This part has been done for you.

In [ ]:
from skimage.measure import marching_cubes

verts, faces, _, _ = marching_cubes(pred_vals.reshape([resolution, resolution, resolution]), level=0)
verts = (coords_matrix[:3, :3] @ verts.T + coords_matrix[:3, 3:4]).T

pred_mesh = ps.register_surface_mesh("mesh", verts, faces)
ps.show()

The results should look like that:

<img src="imgs/screenshot_000006.jpg" width="800" />

Please also run this command in order to save renders of 3d images

In [ ]:
!pytest tests.py::test_make_screens

Run tests to check if reconstructions are good enough

In [ ]:
!pytest tests.py::test_3d

----------------

First generate images of 3D models:
```
pytest tests.py::test_make_screens
```
Then we can add them:
```
git add bonus_img_saves
```

Now you can save and submit your work using git.
To trigger auto-grading, you need to push your changes to git:

```
git add .
git commit -m "update"
git push origin main
```

After you pushed to github, an **automatic test** will start.
You can see the result at the `Actions` tab on your github repo page.